In [1]:
from pathlib import Path

import pandas as pd

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results_path = Path("~/ray_results").expanduser()
assert ray_results_path.is_dir()

In [3]:
last_experiment = max(ray_results_path.glob('TrainLightGCN_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

class TrainLightGCN:
    def __init__(self):
        raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), TrainLightGCN)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/daviddavo/ray_results/TrainLightGCN_2023-11-10_20-19-58...


2023-11-12 08:36:08,618	WARNING experiment_analysis.py:205 -- Failed to fetch metrics for 1 trial(s):
- TrainLightGCN_d6fad850: FileNotFoundError('Could not fetch metrics for TrainLightGCN_d6fad850: both result.json and progress.csv were not found at /home/daviddavo/ray_results/TrainLightGCN_2023-11-10_20-19-58/TrainLightGCN_d6fad850_665_trial_index=4,batch_size=6,conv_layers=2,embedding_dim=330,l2=0.0000,learning_rate=0.0004_2023-11-12_08-32-54')


Result(
  metrics={'iteration': 200, 'loss': 3.252770148947094e-05, 'mf_loss': 3.215822999209195e-05, 'emb_loss': 3.694721903002573e-07, 'recall': 0.01652892561983471, 'ndcg': 0.00841142612236363, 'precision': 0.0033057851239669425, 'map': 0.005785123966942148},
  path='/home/daviddavo/ray_results/TrainLightGCN_2023-11-10_20-19-58/TrainLightGCN_330f9874_562_trial_index=1,batch_size=7,conv_layers=2,embedding_dim=425,l2=0.0000,learning_rate=0.0055_2023-11-12_01-17-42',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/TrainLightGCN_2023-11-10_20-19-58/TrainLightGCN_330f9874_562_trial_index=1,batch_size=7,conv_layers=2,embedding_dim=425,l2=0.0000,learning_rate=0.0055_2023-11-12_01-17-42/checkpoint_000000)
)

In [4]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('ndcg', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf[rdf['config/__trial_index__'] >= 3].sort_values('ndcg').tail()

Index(['iteration', 'loss', 'mf_loss', 'emb_loss', 'recall', 'ndcg',
       'precision', 'map', 'done', 'training_iteration', 'trial_id',
       'time_this_iter_s', 'time_total_s', 'checkpoint_dir_name',
       'config/batch_size', 'config/embedding_dim', 'config/conv_layers',
       'config/learning_rate', 'config/l2', 'config/__trial_index__'],
      dtype='object')


,iteration,loss,mf_loss,emb_loss,recall,ndcg,precision,map,done,training_iteration,trial_id,time_this_iter_s,time_total_s,checkpoint_dir_name,config/batch_size,config/embedding_dim,config/conv_layers,config/learning_rate,config/l2,config/__trial_index__
318,105,0.006941,0.002070,0.004872,0.406944,0.309124,0.105000,0.269954,False,21,b6ee6993,53.253463,1134.402971,None,128,280,2,0.0039,1.030567e-04,8
178,85,0.007556,0.007472,0.000084,0.395833,0.327644,0.103333,0.296319,False,17,9be4a2cf,252.565701,3880.706176,None,16,160,4,0.0056,7.808156e-08,8
308,25,0.064241,0.064230,0.000011,0.433333,0.342303,0.115000,0.297986,False,5,50ed6d8d,111.969949,561.750367,None,64,445,3,0.0186,3.551980e-09,8
208,85,2.229349,1.859426,0.369923,0.440278,0.358634,0.115000,0.321319,False,17,da09ab5f,242.635288,4021.178558,None,16,160,4,0.0808,3.901069e-06,8
328,135,1.743646,0.174567,1.569080,0.526389,0.385964,0.143333,0.325509,False,27,03dd757a,323.329594,7097.430680,None,16,190,3,0.0843,4.672307e-03,8


In [5]:
config_cols = [ c for c in rdf.columns if c.startswith('config/') and c != 'config/__trial_index__' ]
print(config_cols)
print(len(rdf.groupby(config_cols)), 'configurations tested')

rdf.groupby(config_cols).mean(numeric_only=True).sort_values('ndcg').tail()

['config/batch_size', 'config/embedding_dim', 'config/conv_layers', 'config/learning_rate', 'config/l2']
67 configurations tested


iteration  \
config/batch_size config/embedding_dim config/conv_layers config/learning_rate config/l2                 
32                495                  5                  0.0166               1.934262e-08       91.5   
                  290                  3                  0.0046               2.202340e-09       74.0   
16                160                  4                  0.0808               3.901069e-06       88.5   
32                60                   2                  0.9110               8.847247e-09       99.5   
128               285                  2                  0.0373               1.509431e-08      110.5   

                                                                                                   loss  \
config/batch_size config/embedding_dim config/conv_layers config/learning_rate config/l2                  
32                495                  5                  0.0166               1.934262e-08    0.049101   
                  290                  3                  0.0046               2.202340e-09    0.006763   
16                160                  4                  0.0808               3.901069e-06    1.389348   
32                60                   2                  0.9110               8.847247e-09  190.539756   
128               285                  2                  0.0373               1.509431e-08    0.122773   

                                                                                                mf_loss  \
config/batch_size config/embedding_dim config/conv_layers config/learning_rate config/l2                  
32                495                  5                  0.0166               1.934262e-08    0.048983   
                  290                  3                  0.0046               2.202340e-09    0.006762   
16                160                  4                  0.0808               3.901069e-06    1.159179   
32                60                   2                  0.9110               8.847247e-09  190.504621   
128               285                  2                  0.0373               1.509431e-08    0.122639   

                                                                                                 emb_loss  \
config/batch_size config/embedding_dim config/conv_layers config/learning_rate config/l2                    
32                495                  5                  0.0166               1.934262e-08  1.186507e-04   
                  290                  3                  0.0046               2.202340e-09  9.678145e-07   
16                160                  4                  0.0808               3.901069e-06  2.301686e-01   
32                60                   2                  0.9110               8.847247e-09  3.513562e-02   
128               285                  2                  0.0373               1.509431e-08  1.343473e-04   

                                                                                               recall  \
config/batch_size config/embedding_dim config/conv_layers config/learning_rate config/l2                
32                495                  5                  0.0166               1.934262e-08  0.197062   
                  290                  3                  0.0046               2.202340e-09  0.191907   
16                160                  4                  0.0808               3.901069e-06  0.212511   
32                60                   2                  0.9110               8.847247e-09  0.194055   
128               285                  2                  0.0373               1.509431e-08  0.194065   

                                                                                                 ndcg  \
config/batch_size config/embedding_dim config/conv_layers config/learning_rate config/l2                
32                495                  5                  0.0166               1.934262e-08  0.134389   
                  290              

In [6]:
ORG_NAME = 'Decentraland'
N_SPLITS = 10

bdf = pd.read_csv(f'./data/baseline-mp-{ORG_NAME}-{N_SPLITS}.csv')
bdf.mean()

precision@5     0.160523
ndcg@5          0.317748
map@5           0.247825
precision@10    0.145554
ndcg@10         0.420900
map@10          0.308240
dtype: float64